In [50]:
import pandas as pd
import sqlite3

# Load CSV files
prods = pd.read_csv("PRODUCTS_TAKEHOME.csv")
trans = pd.read_csv("TRANSACTION_TAKEHOME.csv")
users = pd.read_csv("USER_TAKEHOME.csv")



# Set up in-memory SQLite DB
conn = sqlite3.connect(":memory:")

# Load DataFrames into SQLite tables
prods.to_sql("products", conn, index=False, if_exists="replace")
trans.to_sql("transactions", conn, index=False, if_exists="replace")
users.to_sql("users", conn, index=False, if_exists="replace")

# Cleaning product data

conn.execute( """CREATE TABLE products_cleaned AS 
SELECT CAST( barcode AS TEXT) AS cleanBarcode, CATEGORY_1, CATEGORY_2, CATEGORY_3, CATEGORY_4, BRAND
FROM products
WHERE BARCODE IS NOT NULL
GROUP BY CATEGORY_1, CATEGORY_2, CATEGORY_3, CATEGORY_4, BRAND""") 

conn.commit()

query = """ SELECT CATEGORY_1, CATEGORY_2, CATEGORY_3, CATEGORY_4
FROM products_cleaned
WHERE CATEGORY_2 like '%Dips & Salsa%'
GROUP BY CATEGORY_1, CATEGORY_2, CATEGORY_3, CATEGORY_4
ORDER BY CATEGORY_2
LIMIT 100
"""


df = pd.read_sql_query(query, conn)
print(df)


   CATEGORY_1    CATEGORY_2        CATEGORY_3 CATEGORY_4
0      Snacks  Dips & Salsa              None       None
1      Snacks  Dips & Salsa          Bean Dip       None
2      Snacks  Dips & Salsa        Cheese Dip       None
3      Snacks  Dips & Salsa      Dessert Dips       None
4      Snacks  Dips & Salsa         Dip Mixes       None
5      Snacks  Dips & Salsa  French Onion Dip       None
6      Snacks  Dips & Salsa         Guacamole       None
7      Snacks  Dips & Salsa            Hummus       None
8      Snacks  Dips & Salsa        Other Dips       None
9      Snacks  Dips & Salsa         Ranch Dip       None
10     Snacks  Dips & Salsa             Salsa       None
